In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
# part 1 : crawling the website for each volume

my_url = 'https://www.tandfonline.com/loi/reus20'
response = requests.get(url=my_url)

# check if the request works or not, 200 if all goes well
print(response)

web_content = response.content

<Response [200]>


In [3]:
bs_obj=BeautifulSoup(web_content,'lxml')
# print(bs_obj)
# bs_obj.title
bs_obj.title.string

'List of issues European Societies'

In [81]:
div_obj = bs_obj.body.div.div

# for i in div_obj.children:
#     print(i)

# for i in div_obj.contents:
#     print(i)

# div_obj.children

volume = div_obj.find_all(name='li', attrs={"class":"vol_li"})
# volume
# print(type(div_obj))
# print(type(volume))

volume_df = pd.DataFrame(columns=["number_volume","year","website_volume"])

for i in range(len(volume)):
    
    date = volume[i].find(name='h3').string.split('\n')[1].split(' ') 
    number = date[1]
    year = date[2]
    
    website = str(volume[i].find(name='a')).split('"')[3]
    website = 'https://www.tandfonline.com' + website
    
    volume_df.loc[i] = [number, year, website]

volume_df = volume_df.iloc[::-1]
volume_df.reset_index(drop=True,inplace=True)

# print(os.getcwd())
path = os.path.abspath(os.path.dirname(os.getcwd())) + '\\data'

volume_df.to_csv(path+'\\website\\volume.csv',index=False,sep=',')

volume_df.head()

,number_volume,year,website_volume
0,1,1999,https://www.tandfonline.com/loi/reus20?treeId=...
1,2,2000,https://www.tandfonline.com/loi/reus20?treeId=...
2,3,2001,https://www.tandfonline.com/loi/reus20?treeId=...
3,4,2002,https://www.tandfonline.com/loi/reus20?treeId=...
4,5,2003,https://www.tandfonline.com/loi/reus20?treeId=...


In [179]:
# part 2 : crawling the website for each volume

volume_df = pd.read_csv(path+'\\website\\volume.csv')

def find_website_issue(i):
    
    number_volume = volume_df['number_volume'].loc[i]
    year = volume_df['year'].loc[i]
    url_volume = volume_df['website_volume'].loc[i]
    
    response = requests.get(url=url_volume)
    # print(response)
    web_content = response.content
    
    bs_obj=BeautifulSoup(web_content,'lxml')
    # print(bs_obj.title.string)
    
    div_obj = bs_obj.body.div.div
    issue = div_obj.find_all(name='a', attrs={"class":"issue-link"})
    
    issue_df = pd.DataFrame(columns=["number_volume", "number_issue", "year",
                                     "page_start","page_end","website_issue"])
    
    for i in range(len(issue)):
        
        number_issue = issue[i].find(name="div").string.split(' ')[1]
        
        page = issue[i].find(name='span', attrs={"class":"loiIssuePages"}).string.split('\n')[1].split(' ')[-1].split('-')
        page_start = page[0]
        page_end = page[1]
        
        website = str(issue[i]).split('>')[0].split('"')[-2]
        website = 'https://www.tandfonline.com' + website
        
        issue_df.loc[i] = [number_volume, number_issue, year, page_start, page_end, website]
        
    issue_df = issue_df.iloc[::-1]
    issue_df.reset_index(drop=True,inplace=True)
    
    return issue_df


In [207]:
path = os.path.abspath(os.path.dirname(os.getcwd())) + '\\data'

num_vol = volume_df.shape[0]
for i_vol in range(num_vol):

    issue_df = find_website_issue(i_vol)
    
    issue_df.to_csv(path+'\\website\\volume_'+str(i_vol)+'.csv',index=False,sep=',')

In [4]:
# part 3 : put all the webistes into one form

path = os.path.abspath(os.path.dirname(os.getcwd())) + '\\data'

volume_df = pd.read_csv(path+'\\website\\volume.csv')
num_vol = volume_df.shape[0]

issue_df_all = pd.DataFrame(columns=["number_volume", "number_issue", "year",
                                     "page_start","page_end","website_issue"])

for i_vol in range(num_vol):

    issue_df = pd.read_csv(path+'\\website\\volume_'+str(i_vol)+'.csv')
    
    issue_df_all = pd.concat([issue_df_all, issue_df], axis=0, ignore_index=True)

In [6]:
issue_df_all.to_csv(path+'\\website\\issue.csv',index=False,sep=',')